In [0]:
import os
import pandas as pd
import tqdm
from tqdm import tqdm_notebook
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [0]:

df_loan = pd.read_csv( "loan.csv")
df_loan.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,NaN,3,0,13648,83.70%,9,f,0.00,0.00,5863.155187,5833.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,NaN,3,0,1687,9.40%,4,f,0.00,0.00,1008.710000,1008.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,NaN,2,0,2956,98.50%,10,f,0.00,0.00,3005.666844,3005.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0,Feb-96,1,35.0,NaN,10,0,5598,21%,37,f,0.00,0.00,12231.890000,12231.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0,Jan-96,0,38.0,NaN,15,0,27783,53.90%,38,f,524.06,524.06,3513.330000,3513.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [0]:
df_loan.shape

(39717, 111)

In [0]:
Coloums = df_loan.columns
Coloums

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)

In [0]:
print("checking if any null values are present\n", df_loan.isna().sum())

checking if any null values are present
 id                                    0
member_id                             0
loan_amnt                             0
funded_amnt                           0
funded_amnt_inv                       0
term                                  0
int_rate                              0
installment                           0
grade                                 0
sub_grade                             0
emp_title                          2459
emp_length                         1075
home_ownership                        0
annual_inc                            0
verification_status                   0
issue_d                               0
loan_status                           0
pymnt_plan                            0
url                                   0
desc                              12940
purpose                               0
title                                11
zip_code                              0
addr_state                            0

* Minimum number of NaNs in the coloumns is 3/4 of the Total Data*

In [0]:
df_loan = df_loan.dropna(thresh = (df_loan.shape[0]*(3/4)), axis = 1)

In [0]:
df_loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,3,0,13648,83.70%,9,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,car,bike,309xx,GA,1.00,0,Apr-99,5,3,0,1687,9.40%,4,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-13,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,2,0,2956,98.50%,10,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,other,personel,917xx,CA,20.00,0,Feb-96,1,10,0,5598,21%,37,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,Apr-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://lendingclub.com/browse/loanDetail.acti...,other,Personal,972xx,OR,17.94,0,Jan-96,0,15,0,27783,53.90%,38,f,524.06,524.06,3513.330000,3513.33,2475.94,1037.39,0.00,0.00,0.00,May-16,67.79,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0


In [0]:
df_loan.shape

(39717, 53)

In [0]:
print("checking if any null values are present\n", df_loan.isna().sum())

checking if any null values are present
 id                               0
member_id                        0
loan_amnt                        0
funded_amnt                      0
funded_amnt_inv                  0
term                             0
int_rate                         0
installment                      0
grade                            0
sub_grade                        0
emp_title                     2459
emp_length                    1075
home_ownership                   0
annual_inc                       0
verification_status              0
issue_d                          0
loan_status                      0
pymnt_plan                       0
url                              0
purpose                          0
title                           11
zip_code                         0
addr_state                       0
dti                              0
delinq_2yrs                      0
earliest_cr_line                 0
inq_last_6mths                   0
open_acc      

In [0]:
 df_String_Columns = df_loan.select_dtypes(include=['object'])
                                      

In [0]:
 df_String_Columns.columns

Index(['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'purpose', 'title', 'zip_code', 'addr_state',
       'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d',
       'last_credit_pull_d', 'application_type'],
      dtype='object')

In [0]:
 df_String_Columns.count()

term                   39717
int_rate               39717
grade                  39717
sub_grade              39717
emp_title              37258
emp_length             38642
home_ownership         39717
verification_status    39717
issue_d                39717
loan_status            39717
pymnt_plan             39717
url                    39717
purpose                39717
title                  39706
zip_code               39717
addr_state             39717
earliest_cr_line       39717
revol_util             39667
initial_list_status    39717
last_pymnt_d           39646
last_credit_pull_d     39715
application_type       39717
dtype: int64

 **Convert Load Status to class variables from string**

In [0]:
#loan_status_dict = { 'Fully Paid': 0,  'Charged Off': 1, 'Current': 2}
df_loan['loan_status'] = df_loan['loan_status'].map( { 'Fully Paid': 0,  'Charged Off': 1, 'Current': 2})


**Convert  string features into Numerics**

In [0]:
df_loan['verification_status'] = df_loan['verification_status'].map({'Verified':2,'Source Verified':1,'Not Verified':0})

In [0]:
df_loan['revol_util'] = df_loan['revol_util'].fillna('0.0%')

df_loan['revol_util'] = df_loan['revol_util'].apply(lambda x:float(x[:-1]) )

In [0]:
df_loan['int_rate'] = df_loan['int_rate'].apply(lambda x:float(x[:-1]) )

In [0]:
df_loan['grade'] = df_loan['grade'].map({'A':7,'B':6,'C':5,'D':4,'E':3,'F':2,'G':1})

In [0]:
df_loan["home_ownership"] = df_loan["home_ownership"].map({"MORTGAGE":5,"RENT":4,"OWN":3,"OTHER":2,"NONE":1})


In [0]:
df_loan['emp_length'] = df_loan['emp_length'].fillna(0)
df_loan['emp_length'] = df_loan['emp_length'] .replace({'years':'','year':'',' ':'','<':'','\+':''}, regex = True)
df_loan['emp_length']= df_loan['emp_length'].apply(lambda x:int(x))

In [0]:
df_loan['term'] = df_loan['term'].fillna(0)
df_loan['term'] = df_loan['term'] .replace({'months':'',' ':''}, regex = True)
df_loan['term']= df_loan['term'].apply(lambda x:int(x))

In [0]:
df_loan

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599,5000,5000,4975.00000,36,10.65,162.87,6,B2,NaN,10,4,24000.00,2,Dec-11,0,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,3,0,13648,83.70,9,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.0000,Jan-15,171.62,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
1,1077430,1314167,2500,2500,2500.00000,60,15.27,59.83,5,C4,Ryder,1,4,30000.00,1,Dec-11,1,n,https://lendingclub.com/browse/loanDetail.acti...,car,bike,309xx,GA,1.00,0,Apr-99,5,3,0,1687,9.40,4,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.1100,Apr-13,119.66,Sep-13,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
2,1077175,1313524,2400,2400,2400.00000,36,15.96,84.33,5,C5,NaN,10,4,12252.00,0,Dec-11,0,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,2,0,2956,98.50,10,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.0000,Jun-14,649.91,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
3,1076863,1277178,10000,10000,10000.00000,36,13.49,339.31,5,C1,AIR RESOURCES BOARD,10,4,49200.00,1,Dec-11,0,n,https://lendingclub.com/browse/loanDetail.acti...,other,personel,917xx,CA,20.00,0,Feb-96,1,10,0,5598,21.00,37,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.0000,Jan-15,357.48,Apr-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
4,1075358,1311748,3000,3000,3000.00000,60,12.69,67.79,6,B5,University Medical Group,1,4,80000.00,1,Dec-11,2,n,https://lendingclub.com/browse/loanDetail.acti...,other,Personal,972xx,OR,17.94,0,Jan-96,0,15,0,27783,53.90,38,f,524.06,524.06,3513.330000,3513.33,2475.94,1037.39,0.00,0.00,0.0000,May-16,67.79,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
5,1075269,1311441,5000,5000,5000.00000,36,7.90,156.46,7,A4,Veolia Transportaton,3,4,36000.00,1,Dec-11,0,n,https://lendingclub.com/browse/loanDetail.acti...,wedding,My wedding loan I promise to pay back,852xx,AZ,11.20,0,Nov-04,3,9,0,7963,28.30,12,f,0.00,0.00,5632.210000,5632.21,5000.00,632.21,0.00,0.00,0.0000,Jan-15,161.03,Jan-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
6,1069639,1304742,7000,7000,7000.00000,60,15.96,170.08,5,C5,Southern Star Photography,8,4,47004.00,0,Dec-11,0,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Loan,280xx,NC,23.51,0,Jul-05,1,7,0,17726,85.60,11,f,0.00,0.00,10110.840000,10110.84,6985.61,3125.23,0.00,0.00,0.0000,May-16,1313.76,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
7,1072053,1288686,3000,3000,3000.00000,36,18.64,109.43,3,E1,MKC Accounting,9,4,48000.00,1,Dec-11,0,n,https://lendingclub.com/browse/loanDetail.acti...,car,Car Downpayment,900xx,CA,5.35,0,Jan-07,2,4,0,8221,87.50,4,f,0.00,0.00,3939.135294,3939.14,3000.00,939.14,0.00,0.00,0.0000,Jan-15,111.34,Dec-14,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
8,1071795,1306957,5600,5600,5600.00000,60,21.28,152.39,2,F2,NaN,4,3,40000.00,1,Dec-11,1,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,Expand Business & Buy Debt Portfolio,958xx,CA,5.55,0,Apr-04,2,11,0,5210,32.60,13,f,0.00,0.00,646.020000,646.02,162.02,294.94,0.00,189.06,2.0900,Apr-12,152.39,Aug-12,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
9,1071570,1306721,5375,5375,5350.00000,60,12.69,121.45,6,B5,Starbucks,1,4,15000.00,2,Dec-11,1,n,https://lendingclub.com/browse/loanDetail.acti...,other,Building my credit history.,774xx,TX,18.08,0,Sep-04,0,2,0,9279,36.50,3,f,0.00,0.00,1476.190000,1469.34,673.48,533.42,0.00

**Find All Numerics Coloumns in the Processed Data**

In [0]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_Numerics_Columns = df_loan.select_dtypes(include=numerics)

In [0]:
# Number of Coloumsn with Numerics 
df_Numerics_Columns.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In [0]:
# Replace NaNs with mean vlaue
for i in range(0,len(df_Numerics_Columns.columns)):
  df_loan[df_Numerics_Columns.columns[i]] = df_loan[df_Numerics_Columns.columns[i]].fillna(df_loan[df_Numerics_Columns.columns[i]].mean())
 
  

In [0]:
print("checking if any null values are present\n", df_loan[df_Numerics_Columns.columns].isna().sum())

checking if any null values are present
 id                            0
member_id                     0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
int_rate                      0
installment                   0
grade                         0
emp_length                    0
home_ownership                0
annual_inc                    0
verification_status           0
loan_status                   0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
out_prncp                     0
out_prncp_inv                 0
total_pymnt                   0
total_pymnt_inv               0
total_rec_prncp               0
total_rec_int                 0
total_rec_late_fee            0
recoveries                    0

**Replace NaNs with Zeros **

In [0]:
df_loan_1 = df_loan[df_Numerics_Columns.columns]

In [0]:
corr_matrix = df_loan_1.corr()
corr_matrix['loan_status'].sort_values(ascending=False)

loan_status                   1.000000
out_prncp                     0.528067
out_prncp_inv                 0.527508
term                          0.313396
int_rate                      0.246591
recoveries                    0.237785
total_rec_int                 0.222922
id                            0.154953
member_id                     0.148226
collection_recovery_fee       0.143521
loan_amnt                     0.132566
funded_amnt                   0.129093
funded_amnt_inv               0.122327
total_rec_late_fee            0.119011
verification_status           0.093135
revol_util                    0.090518
installment                   0.058768
dti                           0.057713
inq_last_6mths                0.043814
emp_length                    0.037607
pub_rec                       0.031822
pub_rec_bankruptcies          0.030715
revol_bal                     0.026956
delinq_2yrs                   0.013581
open_acc                      0.009954
home_ownership           

**Selection of the features for the model**

In [0]:
features = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc',  'dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
        'pub_rec_bankruptcies','verification_status','revol_util','loan_status']
 

**Creating an ew data frames for the new features**

In [0]:
df_loan_2=df_loan_1[features]

In [0]:
# Count Total number of units in each class
df_loan_2['loan_status'].value_counts() 

0    32950
1     5627
2     1140
Name: loan_status, dtype: int64

**In order to elimate model to have selection bias to one catageory need to select same amount of data from each catagory**

In [0]:
# Selecting the random 1140 samples from each data category and Contacate the all three cateogy to form new data frame
loanstatus_0 = df_loan_2[df_loan_2["loan_status"]==0]
loanstatus_1 = df_loan_2[df_loan_2["loan_status"]==1]
loanstatus_2 = df_loan_2[df_loan_2["loan_status"]==2]
subset_of_loanstatus_0 = loanstatus_0.sample(n=1140)
subset_of_loanstatus_1 = loanstatus_1.sample(n=1140)
subset_of_loanstatus_2 = loanstatus_2.sample(n=1140)
data = pd.concat([subset_of_loanstatus_2,subset_of_loanstatus_1, subset_of_loanstatus_0])
data = data.sample(frac=1).reset_index(drop=True)
print("Current shape of dataset :",data.shape)
data.head()

Current shape of dataset : (3420, 31)


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies,verification_status,revol_util,loan_status
0,15000,15000,11547.09061,36,14.50,516.32,3,8,4,74784.0,24.61,0,3,17,0,55632,21,0.0,0.0,18619.964920,14333.92,15000.00,3594.14,25.82,0.00,0.0000,1049.64,0.0,0,80.7,0
1,5000,5000,5000.00000,36,12.69,167.73,6,4,4,32000.0,9.38,0,2,9,0,5532,11,0.0,0.0,3835.660000,3835.66,2677.58,835.09,0.00,322.99,3.2299,167.73,0.0,1,53.2,1
2,4200,4200,4075.00000,36,7.51,130.66,7,2,4,40000.0,15.57,0,2,6,0,1801,12,0.0,0.0,4703.910242,4563.91,4200.00,503.91,0.00,0.00,0.0000,148.47,0.0,0,62.1,0
3,13800,13800,13800.00000,36,8.49,435.57,7,2,4,79000.0,14.22,0,0,9,0,23137,15,0.0,0.0,14375.202100,14375.20,13800.00,575.20,0.00,0.00,0.0000,10152.07,0.0,0,68.9,0
4,10000,10000,10000.00000,36,7.49,311.02,7,2,5,28800.0,10.42,0,0,9,0,468,26,0.0,0.0,11177.400010,11177.40,10000.00,1177.40,0.00,0.00,0.0000,1560.85,0.0,1,6.8,0


In [0]:
X, Y = data.iloc[:,:-1].values, data.iloc[:,-1].values

In [0]:
X.shape

(3420, 30)

In [0]:
Y.shape

(3420,)

**Normalizing the input features**

In [0]:
#Min 
a = 0
# Max
b = 1
# Function for narmalizing the Min and Max
def Normalization_Transform(x, a,b):
    x_min=np.amin(x, axis=0)
    x_max=np.amax(x, axis=0)
    xn = a +((x-x_min) *( (b-a)/(x_max - x_min)))
    return xn, x_min, x_max

In [0]:
def Normalization_Test_Transform(x,x_min,x_max, a,b):
   
    xn = a +((x-x_min) *( (b-a)/(x_max - x_min)))
    return xn

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [0]:
X_train.shape

(2736, 30)

In [0]:
X_test.shape

(684, 30)

In [0]:
X_train_Norm,X_min, X_max = Normalization_Transform(X_train,a,b)
X_test_Norm=  Normalization_Test_Transform( X_test,X_min, X_max,a,b)

In [0]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [0]:
LR = LogisticRegression(C= 1000,penalty= 'l1').fit(X_train_Norm, y_train)
predictproba_LR =LR.predict(X_test_Norm)
print("Training  set LR Accuracy",round(LR.score(X_train_Norm, y_train), 4)*100)
print("Testing  set  LR Accuracy",round(LR.score(X_test_Norm,y_test), 4)*100)



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Training  set LR Accuracy 99.85000000000001
Testing  set  LR Accuracy 99.27


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:

clf_svm = svm.SVC(C = 100,gamma='scale',decision_function_shape="ovo",kernel= 'linear').fit(X_train_Norm, y_train)
predictproba_SVM = clf_svm.predict(X_test_Norm)
print("Training SVM Accuracy",round(clf_svm.score(X_train_Norm, y_train), 4)*100)
print("Testing  SVM Accuracy",round(clf_svm.score(X_test_Norm, y_test), 4)*100)



Training SVM Accuracy 99.2
Testing  SVM Accuracy 98.39


In [0]:
RF = RandomForestClassifier(bootstrap=True, max_depth= 30, n_estimators=200, random_state=0).fit(X_train_Norm, y_train)

predictproba_RF = RF.predict(X_test_Norm)
print("Training Random Forest  Accuracy",round(RF.score(X_train_Norm, y_train), 4)*100)
print("Testing  Random Forest  Accuracy",round(RF.score(X_test_Norm,y_test), 4)*100)




Training Random Forest  Accuracy 100.0
Testing  Random Forest  Accuracy 99.27


In [0]:
NN = MLPClassifier(solver='lbfgs', alpha=1e-3, max_iter= 1900, hidden_layer_sizes=14,random_state=0).fit(X_train_Norm, y_train)

predictproba_NN = NN.predict(X_test_Norm)
print("Training Neural Networks  Accuracy",round(NN.score(X_train_Norm, y_train), 4)*100)
print("Testing  Neural Networks  Accuracy",round(NN.score(X_test_Norm,y_test), 4)*100)



Training Neural Networks  Accuracy 100.0
Testing  Neural Networks  Accuracy 97.81


In [0]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7,metric='manhattan',weights = 'distance').fit(X_train_Norm, y_train)
predictproba_KNN = neigh.predict(X_test_Norm)
print("Training K Nearest Neighbour",round(neigh.score(X_train_Norm, y_train), 4)*100)
print("Testing  K Nearest Neighbour",round(neigh.score(X_test_Norm,y_test), 4)*100)

Training K Nearest Neighbour 100.0
Testing  K Nearest Neighbour 84.36


**Precision,recall,F1score for all models**

In [0]:

from sklearn import metrics

print("LR",metrics.classification_report(y_test, predictproba_LR, digits=3))
print("SVM",metrics.classification_report(y_test,predictproba_SVM , digits=3))
print("RF",metrics.classification_report(y_test,predictproba_RF,digits=3))
print("MLP",metrics.classification_report(y_test, predictproba_NN, digits=3))
print("KNN",metrics.classification_report(y_test, predictproba_KNN, digits=3))

LR               precision    recall  f1-score   support

           0      0.984     1.000     0.992       243
           1      0.996     0.987     0.991       226
           2      1.000     0.991     0.995       215

    accuracy                          0.993       684
   macro avg      0.993     0.992     0.993       684
weighted avg      0.993     0.993     0.993       684

SVM               precision    recall  f1-score   support

           0      0.957     1.000     0.978       243
           1      1.000     0.965     0.982       226
           2      1.000     0.986     0.993       215

    accuracy                          0.984       684
   macro avg      0.986     0.984     0.984       684
weighted avg      0.985     0.984     0.984       684

RF               precision    recall  f1-score   support

           0      0.980     1.000     0.990       243
           1      1.000     0.978     0.989       226
           2      1.000     1.000     1.000       215

    accura